In [ ]:
%matplotlib inline
import random
import numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
import cv2
import yaml

from bstld.tf_object_detection.inference import label_file_inference
from bstld.evaluation.evaluate import evaluate
from bstld.tf_object_detection import constants

# Data preprocessing

fix bug with paths in test.yaml

In [ ]:
!cp data/test.yaml test.yaml
!sed -i 's/\/net\/pal-soc1.us.bosch.com\/ifs\/data\/Shared_Exports\/deep_learning_data\/traffic_lights\/university_run1/.\/data\/rgb\/test/g' test.yaml

create tfrecords to speedup training

In [ ]:
! python3 bstld/tf_object_detection/to_tfrecords.py \
   --dataset_folder data \
   --train_yaml data/train.yaml \
   --test_yaml test.yaml \
   --additional_yaml data/additional_train.yaml

# Train a model

Skip this block if you want to use pretrained model

In [ ]:
! python3 /root/models/research/object_detection/legacy/train.py \
  -train_dir=trained -pipeline_config_path=ssd_mobilenet_v1.config

# Evaluate a model

Few helper functions

In [ ]:
def preprocess_gt(gt):
    res = []
    scale_x = lambda x: x / constants.WIDTH
    scale_y = lambda y: y / constants.HEIGHT
    map_label = lambda l: constants.EVAL_ID_MAP[
        constants.SIMPLIFIED_CLASSES[l]]
    for obj in gt:
        objs = [(x['x_min'], x['x_max'], x['y_min'], x['y_max'], x['label'])
                for x in obj['boxes'] if not x['occluded']]
        if len(objs) == 0:
            res.append({'xmin':[], 'ymin':[], 'xmax':[], 'ymax':[],
                        'class_labels':[], 'file': obj['path']})
            continue
        xmin, xmax, ymin, ymax, label = zip(*objs)
        xmin = list(map(scale_x, xmin))
        xmax = list(map(scale_x, xmax))
        ymin = list(map(scale_y, ymin))
        ymax = list(map(scale_y, ymax))
        label = list(map(map_label, label))
        res.append({'file': obj['path'], 'xmin': xmin, 'xmax': xmax,
                    'ymin': ymin, 'ymax': ymax, 'class_labels': label})
    return res

def preprocess_results(results):
    for res in results:
        for k in ['detection_classes', 'detection_scores', 'detection_boxes']:
            res[k] = res[k][0]
        res['detection_classes'] -= 1
        
def draw_gt(img, gt):
    keys = ['xmin', 'ymin', 'xmax', 'ymax', 'class_labels']
    gt = [gt[k] for k in keys]
    for xmin, ymin, xmax, ymax, c in list(zip(*gt)):
        c = constants.CLASS_COLORS[c+1]
        xmin, xmax = map(lambda x: int(x * constants.WIDTH), [xmin, xmax])
        ymin, ymax = map(lambda x: int(x * constants.HEIGHT), [ymin, ymax])
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), c, 4)
        
def draw_det(img, det, threshold=0.5):
    b = det['detection_boxes']
    c = det['detection_classes']
    s = det['detection_scores']
    det = zip(*(list(zip(*b)) + [c, s]))
    for ymin, xmin, ymax, xmax, c, s in det:
        if s < threshold:
            continue
        c = constants.CLASS_COLORS[c+1]
        xmin, xmax = map(lambda x: int(x * constants.WIDTH), [xmin, xmax])
        ymin, ymax = map(lambda x: int(x * constants.HEIGHT), [ymin, ymax])
        cv2.rectangle(img, (xmin, ymin), (xmax, ymax), c, 4)
        
def show_plots(res):
    plt.figure(1)
    plt.suptitle('BSTLD precision/recall curves')
    for i, key in enumerate(res['precisions']):
        if len(res['precisions']) > 1:
            plt.subplot(int('22' + str(i + 1)))  # 22 grid structure, i+1 index
        plt.ylim((0, 1))
        plt.xlim((0, 1))
        plt.xlabel('recall')
        plt.ylabel('precision')
        plt.grid()
        plt.plot(res['recalls'][key], res['precisions'][key])
        plt.title(key)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

Freeze trained model

In [ ]:
! python3 bstld/tf_object_detection/model_freezer.py \
  --train_dir trained

Apply model

In [ ]:
! cp -r models/model.ckpt-128199_proto trained/

In [ ]:
graph_path = 'trained/model.ckpt-128199_proto/frozen_inference_graph.pb'
data_file = 'test.yaml'
threshold = .05

with open(data_file, 'r') as f:
    gt = yaml.load(f)
gt = preprocess_gt(gt)

results = label_file_inference(graph_path,
                               data_file,
                               json_out_path='',
                               show=0,
                               out_folder=None,
                               threshold=threshold)
preprocess_results(results)

Show results on a random image

In [ ]:
k = random.randrange(len(gt))
plt.rcParams['figure.figsize'] = [10, 5]

img = cv2.imread(gt[k]['file'])
draw_gt(img, gt[k])
plt.title('ground truth')
plt.imshow(img[...,::-1])
plt.show()

plt.title('detector results')
img = cv2.imread(gt[k]['file'])
draw_det(img, results[k], 0.5)
plt.imshow(img[...,::-1])
plt.show()

compute metrics

In [ ]:
res = evaluate(gt, results, "res.json")
print('Mean AP value {:.3f}'.format(res['mean_ap']))
print('Weighet mean AP value {:.3f}'.format(res['weighted_mean_ap']))